In [6]:
import ROOT
import numpy as np

In [7]:
#--------------------------------
# simulate some data
#--------------------------------
background_data = np.random.gamma(shape=2.2, scale=20, size=90000)
signal_data = np.random.normal(loc=100, scale=12, size=10000)
data_values = np.concatenate((signal_data, background_data))


In [8]:
#--------------------------------
# make model
#-------------------------------
# this is the distribution we'll be looking at
mass = ROOT.RooRealVar("mass", "mass", 0, 200)

In [9]:
# background variables
gamma = ROOT.RooRealVar("gamma", "gamma", 1, 0, 1e6)
beta = ROOT.RooRealVar("beta", "beta",    1, 0, 1e6)
mu = ROOT.RooRealVar("mu", "mu",          0, 0, 0) # fix this at 0

# signal variables
# we want to find a particle in the 60 --> 140 GeV range
mean = ROOT.RooRealVar("mean", "mean",    80, 60, 140)
sigma = ROOT.RooRealVar("sigma", "sigma", 1, 0, 40)

In [10]:
# what fraction is signal
# initial guess = half
fsig = ROOT.RooRealVar("fsig", "fsig", 0.5, 0, 1)

# background
sig = ROOT.RooGaussian("sig", "sig",
                       mass, mean, sigma)
# background
bkg = ROOT.RooGamma("bkg", "bkg",
                    mass, gamma, beta, mu)

# total model is the sum of those
model = ROOT.RooAddPdf("model", "model",
                       ROOT.RooArgList(sig, bkg), fsig)

[#0] WARNING:InputArguments -- The parameter 'sigma' with range [0, 40] of the RooGaussian 'sig' exceeds the safe range of (0, inf). Advise to limit its range.
[#0] WARNING:InputArguments -- The parameter 'gamma' with range [0, 1e+06] of the RooGamma 'bkg' exceeds the safe range of (0, inf). Advise to limit its range.
[#0] WARNING:InputArguments -- The parameter 'beta' with range [0, 1e+06] of the RooGamma 'bkg' exceeds the safe range of (0, inf). Advise to limit its range.


In [11]:
#------------------------------
# fit model to data
#------------------------------
data = ROOT.RooDataSet.from_numpy({'mass': data_values}, [mass])
model.fitTo(data)

# print values
print(mean)
print(sigma)
print(gamma)
print(beta)
print(mu)
print(fsig)

RooRealVar::mean = 99.8946 +/- 0.224619  L(60 - 140) 

RooRealVar::sigma = 11.9429 +/- 0.215895  L(0 - 40) 

RooRealVar::gamma = 2.19386 +/- 0.0105168  L(0 - 1e+06) 

RooRealVar::beta = 20.018 +/- 0.131161  L(0 - 1e+06) 

RooRealVar::mu = 0 +/- 0  L(0 - 0) 

RooRealVar::fsig = 0.10011 +/- 0.0020729  L(0 - 1) 

[#0] WARNING:DataHandling -- RooDataSet.from_numpy() Ignored 73 out-of-range events
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
[#1] INFO:Minimization --  The following expressions will be evaluated in cache-and-track mode: (sig,bkg)
[#1] INFO:Minimization -- RooAbsMinimizerFcn::optimizeConstantTerms: set of constant parameters changed, rerunning const optimizer
[#1] INFO:Minimization --  The following expressions will be evaluated in cache-and-track mode: (sig,bkg)
Minuit2Minimizer: Minimize with max-calls 3000 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 474249.979154882545
Edm   = 2.2

Info in <Minuit2>: Minuit2Minimizer::SetVariable Parameter mu has zero or invalid step size - consider it as constant
Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       4342042.368 Edm =      -947354.6675 NCalls =     23
Info in <Minuit2>: NegativeG2LineSearch Doing a NegativeG2LineSearch since one of the G2 component is negative
Info in <Minuit2>: MnSeedGenerator Negative G2 found - new state: 
  Minimum value : 1313081.138
  Edm           : 2365400.597
  Internal parameters:	[     -1.568796326                0     -1.559008097     0.0954164193     -1.253235898]	
  Internal gradient  :	[     -947748735.8      84932.83865     -15133344.69      13444.79874     -976015.7049]	
  Internal covariance matrix:
[[  4.7607542e-13              0              0              0              0]
 [              0  2.0090225e-05              0              0              0]
 [              0              0 

In [12]:
#-------------------------------
# plot results
#-------------------------------
hist_data = ROOT.TH1F("data_hist", "", 50, 0, 200)
for v in data_values:
    hist_data.Fill(v)

# plot bkg only, sig only, and combo
bkg_only_graph = ROOT.TGraph()
sig_only_graph = ROOT.TGraph()
model_graph =    ROOT.TGraph()

for v in np.linspace(0, 200, 1000):
    mass.setVal(v)
    n = bkg_only_graph.GetN()
    
    bkg_only_graph.SetPoint(n, v, bkg.getVal(mass))
    sig_only_graph.SetPoint(n, v, sig.getVal(mass))
    model_graph.SetPoint(   n, v, model.getVal(mass))


bkg_only_graph.Scale(hist_data.GetBinWidth(1)*len(data_values)*(1.0-fsig.getVal()))
sig_only_graph.Scale(hist_data.GetBinWidth(1)*len(data_values)*fsig.getVal())
model_graph.Scale(hist_data.GetBinWidth(1)*len(data_values))


leg = ROOT.TLegend(0.6, 0.6, 0.88, 0.88)
leg.AddEntry(hist_data, "Data", "lep")
leg.AddEntry(bkg_only_graph, "Background only", "l")
leg.AddEntry(sig_only_graph, "Signal only", "l")
leg.AddEntry(model_graph, "Combined model", "l")


hist_data.GetXaxis().SetTitle("Mass [GeV]")
hist_data.GetYaxis().SetTitle("Events / 4 GeV")
hist_data.SetStats(0)
hist_data.SetLineWidth(2)

bkg_only_graph.SetLineColor(ROOT.kRed)
sig_only_graph.SetLineColor(ROOT.kMagenta)
model_graph.SetLineColor(ROOT.kBlack)


c = ROOT.TCanvas()
hist_data.Draw("e0")
bkg_only_graph.Draw("c, same")
sig_only_graph.Draw("c, same")
model_graph.Draw("c, same")

leg.Draw("same")
c.SaveAs("bump-hunt.png")

Info in <TCanvas::Print>: png file bump-hunt.png has been created
